# Sample QC for GWAS analysis

## As-Is Software Disclaimer

This notebook is delivered "As-Is". Notwithstanding anything to the contrary, DNAnexus will have no warranty, support, liability or other obligations with respect to Materials provided hereunder.

[MIT License](https://github.com/dnanexus/UKB_RAP/blob/main/LICENSE) applies to this notebook.

## Jupyterlab app details (launch configuration)

Recommended configuration
- Runtime: ~5 min
- Cluster configuration: `Single Node`
- Recommended instance: `mem1_ssd1_v2_x36`
- Cost: ~£0.3

## Dependencies

|Library |License|
|:------------- |:-------------|
|[pandas](https://pandas.pydata.org/) |[BSD-3](https://github.com/pandas-dev/pandas/blob/main/LICENSE)|
|[numpy](https://numpy.org/) |[BSD-3](https://github.com/numpy/numpy/blob/main/LICENSE.txt)|

## Introduction

This notebook:
- Loads cohorts created in cohort browser
- Performs sample QC
- Creates a file containing phenotype and covariate information needed for GWAS analysis

Data used for this notebook:
- Cohorts (`ischemia_cases` `ischemia_controls`) created using Cohort Browser
- Dataset to retrieve phenotype data for both cohorts (`ischemia.pheno`)


## Prepare your environment

In [1]:
# Import packages
# dxpy allows python to interact with the platform storage
import dxpy
import numpy as np
import pandas as pd
import re
import subprocess
import glob
import os

In [2]:
imputation_folder = 'Imputation from genotype (GEL)'
imputation_field_id = '21008'
output_dir = '/Data/'

## Load dataset description containing phenotypic data

In [3]:
# Automatically discover dispensed dataset ID
dispensed_dataset = dxpy.find_one_data_object(
    typename='Dataset', name='app*.dataset', folder='/', name_mode='glob'
)
dispensed_dataset_id = dispensed_dataset['id']

In [4]:
# Get project ID
project_id = dxpy.find_one_project()['id']

In [5]:
dataset = (':').join([project_id, dispensed_dataset_id])

Using the `-ddd` parameter will extract 3 dictionary files associated with the dataset.

The 3 dictionary files that are returned include:
1. `entity_dictionary` that contains the different tables resources that are available. The table we’re most interested in tends to be the participant table that contains the information about each participant.
2. `data_dictionary` that contains the different field names that we might want to include in our dataset.
3. `coding_dictionary` that contains a lookup for the values for some of the field names.

In [6]:
# Note: This cell can only be run once. Otherwise, you'll need to delete the existing data tables in order to re-run
cmd = ['dx', 'extract_dataset', dataset, '-ddd', '--delimiter', ',']
subprocess.check_call(cmd)

0

## Load cohorts that were created in cohort browser

Cohorts were created in Cohort Browser. `ischemic_cases` cohort was created by having the following condition in the field [41270](https://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=41270) condition `I20-I25 Ischaemic heart disease`. `ischemic_controls` was created by using Cohorts compare: not in `ischemic_cases`.

Here we use the `load_cohort` function from dxdata to load the cohort record. For more information about dxdata, see the associated [github repository](https://github.com/dnanexus/OpenBio/blob/master/dxdata/getting_started_with_dxdata.ipynb)

In [9]:
# Discover cohort data
dispensed_control_id = list(
    dxpy.find_data_objects(
        typename='CohortBrowser',
        folder='/Cohorts',
        name_mode='exact',
        name='ischemic_controls',
    )
)[0]['id']

dispensed_case_id = list(
    dxpy.find_data_objects(
        typename='CohortBrowser',
        folder='/Cohorts',
        name_mode='exact',
        name='ischemic_cases',
    )
)[0]['id']

In [10]:
control_dataset = (':').join([project_id, dispensed_control_id])
case_dataset = (':').join([project_id, dispensed_case_id])

## Retrieve phenotypic data

Specify fields ID to retrieve, get corresponding UKB RAP field names and print description table.

- `31` - [Sex](https://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=31)
- `2966` - [Age high blood pressure diagnosed](https://biobank.ctsu.ox.ac.uk/crystal/field.cgi?id=2966)
- `22001` - [Genetic sex](https://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=22001)
- `22006` - [Genetic ethnic grouping](https://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=22006)
- `22019` - [Sex chromosome aneuploidy](https://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=22019)
- `22021` - [Genetic kinship to other participants](https://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=22021)
- `21022` - [Age at recruitment](https://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=21022)
- `23104` - [Body mass index (BMI)](https://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=23104)
- `20160` - [Ever smoked](https://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=20160)
- `30760` - [HDL cholesterol](https://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=30760)
- `30780` - [LDL direct](https://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=30780)
- `22020` - [Used in genetic principal components](https://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=22020)
- `22009` - [Genetic principal components](https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=22009)

In [11]:
field_ids = [
    '31',
    '2966',
    '22001',
    '22006',
    '22019',
    '22021',
    '21022',
    '23104',
    '20160',
    '30760',
    '30780',
    '22020',
    '22009'
]

In [12]:
path = os.getcwd()

In [13]:
data_dict_csv = glob.glob(os.path.join(path, '*.data_dictionary.csv'))[0]
data_dict_df = pd.read_csv(data_dict_csv)
data_dict_df.head()

/tmp/ipykernel_978/2068367959.py:2: DtypeWarning: Columns (4,7,8,9,10,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data_dict_df = pd.read_csv(data_dict_csv)


,entity,name,type,primary_key_type,coding_name,concept,description,folder_path,is_multi_select,is_sparse_coding,linkout,longitudinal_axis_type,referenced_entity_field,relationship,title,units
0,participant,eid,string,global,NaN,NaN,NaN,Participant Information,NaN,NaN,NaN,NaN,NaN,NaN,Participant ID,NaN
1,participant,p3_i0,integer,NaN,NaN,NaN,NaN,Assessment centre > Procedural metrics > Proce...,NaN,NaN,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi...,NaN,NaN,NaN,Verbal interview duration | Instance 0,seconds
2,participant,p3_i1,integer,NaN,NaN,NaN,NaN,Assessment centre > Procedural metrics > Proce...,NaN,NaN,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi...,NaN,NaN,NaN,Verbal interview duration | Instance 1,seconds
3,participant,p3_i2,integer,NaN,NaN,NaN,NaN,Assessment centre > Procedural metrics > Proce...,NaN,NaN,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi...,NaN,NaN,NaN,Verbal interview duration | Instance 2,seconds
4,participant,p3_i3,integer,NaN,NaN,NaN,NaN,Assessment centre > Procedural metrics > Proce...,NaN,NaN,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi...,NaN,NaN,NaN,Verbal interview duration | Instance 3,seconds


The UKB participant tables have the following naming convention for the fields (or columns): `p<field id>_i<instance id>_a<array id>`.

In [14]:
def fields_for_id(field_id):
    '''Collect all field names (e.g. 'p<field_id>_iYYY_aZZZ') given a list of field IDs and return string to pass into extract_dataset'''
    field_names = ['eid']
    for _id in field_id:
        select_field_names = list(
            data_dict_df[
                data_dict_df.name.str.match(r'^p{}(_i\d+)?(_a\d+)?$'.format(_id))
            ].name.values
        )
        # Note: This conditional is used to select only the first instance for all fields except '2966'
        # This conditional is not needed otherwise
        # For PCA field, select the first ten PCs
        if _id == '22009':
            field_names += select_field_names[:10]
        # Select only the first instance for all fields except '2966' 
        elif _id != '2966' and len(select_field_names) > 1:
            field_names.append(select_field_names[0])
        else:
            field_names += select_field_names

    field_names = [f'participant.{f}' for f in field_names]
    return ','.join(field_names)

In [15]:
field_names = fields_for_id(field_ids)
field_names

'participant.eid,participant.p31,participant.p2966_i0,participant.p2966_i1,participant.p2966_i2,participant.p2966_i3,participant.p22001,participant.p22006,participant.p22019,participant.p22021,participant.p21022,participant.p23104_i0,participant.p20160_i0,participant.p30760_i0,participant.p30780_i0,participant.p22020,participant.p22009_a1,participant.p22009_a2,participant.p22009_a3,participant.p22009_a4,participant.p22009_a5,participant.p22009_a6,participant.p22009_a7,participant.p22009_a8,participant.p22009_a9,participant.p22009_a10'

### Select phenotypes for case and control cohort

In [16]:
# Load dataset
# Note: This cell can only be run once. Otherwise, you'll need to delete the existing data tables in order to re-run
# Note: There is no space separating the different fields
cmd = [
    'dx',
    'extract_dataset',
    control_dataset,
    '--fields',
    field_names,
    '--delimiter',
    ',',
    '--output',
    'control_dictionary.csv',
]
subprocess.check_call(cmd)

0

In [17]:
cmd = [
    'dx',
    'extract_dataset',
    case_dataset,
    '--fields',
    field_names,
    '--delimiter',
    ',',
    '--output',
    'case_dictionary.csv',
]
subprocess.check_call(cmd)

0

In [18]:
control_dict_csv = 'control_dictionary.csv'
control_df = pd.read_csv(control_dict_csv)
print(control_df.shape)
control_df.head()

(400946, 26)


,participant.eid,participant.p31,participant.p2966_i0,participant.p2966_i1,participant.p2966_i2,participant.p2966_i3,participant.p22001,participant.p22006,participant.p22019,participant.p22021,...,participant.p22009_a1,participant.p22009_a2,participant.p22009_a3,participant.p22009_a4,participant.p22009_a5,participant.p22009_a6,participant.p22009_a7,participant.p22009_a8,participant.p22009_a9,participant.p22009_a10
0,1000128,1,NaN,NaN,NaN,NaN,1.0,1.0,NaN,0.0,...,-12.10990,3.53791,-0.584297,0.604633,-2.46493,-0.448158,3.393600,2.100990,-11.98610,-3.00294
1,1000424,1,NaN,NaN,NaN,NaN,1.0,1.0,NaN,0.0,...,-9.24265,1.92411,0.169679,3.306660,-4.06993,-2.712060,2.389620,-1.731740,2.02853,1.68914
2,1000569,1,NaN,NaN,NaN,NaN,1.0,1.0,NaN,0.0,...,-11.48970,3.69699,-3.154340,-0.964846,-6.10410,-2.444060,-1.613400,-1.357070,4.39817,1.29341
3,1000695,1,NaN,NaN,NaN,NaN,1.0,1.0,NaN,0.0,...,-11.55390,1.22231,-1.271690,0.461033,-4.63990,1.703520,0.433453,-0.659432,1.15092,2.71503
4,1000747,0,NaN,NaN,NaN,NaN,0.0,1.0,NaN,0.0,...,-13.26790,2.05402,-1.896930,3.913610,6.11443,0.617679,0.177710,-0.669613,1.70910,-1.61138


In [19]:
# Rename column headers
control_df = control_df.rename(columns=lambda x: re.sub('participant.', '', x))
control_df.head()

,eid,p31,p2966_i0,p2966_i1,p2966_i2,p2966_i3,p22001,p22006,p22019,p22021,...,p22009_a1,p22009_a2,p22009_a3,p22009_a4,p22009_a5,p22009_a6,p22009_a7,p22009_a8,p22009_a9,p22009_a10
0,1000128,1,NaN,NaN,NaN,NaN,1.0,1.0,NaN,0.0,...,-12.10990,3.53791,-0.584297,0.604633,-2.46493,-0.448158,3.393600,2.100990,-11.98610,-3.00294
1,1000424,1,NaN,NaN,NaN,NaN,1.0,1.0,NaN,0.0,...,-9.24265,1.92411,0.169679,3.306660,-4.06993,-2.712060,2.389620,-1.731740,2.02853,1.68914
2,1000569,1,NaN,NaN,NaN,NaN,1.0,1.0,NaN,0.0,...,-11.48970,3.69699,-3.154340,-0.964846,-6.10410,-2.444060,-1.613400,-1.357070,4.39817,1.29341
3,1000695,1,NaN,NaN,NaN,NaN,1.0,1.0,NaN,0.0,...,-11.55390,1.22231,-1.271690,0.461033,-4.63990,1.703520,0.433453,-0.659432,1.15092,2.71503
4,1000747,0,NaN,NaN,NaN,NaN,0.0,1.0,NaN,0.0,...,-13.26790,2.05402,-1.896930,3.913610,6.11443,0.617679,0.177710,-0.669613,1.70910,-1.61138


In [20]:
case_dict_csv = 'case_dictionary.csv'
case_df = pd.read_csv(case_dict_csv)
print(case_df.shape)
case_df.head()

(91331, 26)


,participant.eid,participant.p31,participant.p2966_i0,participant.p2966_i1,participant.p2966_i2,participant.p2966_i3,participant.p22001,participant.p22006,participant.p22019,participant.p22021,...,participant.p22009_a1,participant.p22009_a2,participant.p22009_a3,participant.p22009_a4,participant.p22009_a5,participant.p22009_a6,participant.p22009_a7,participant.p22009_a8,participant.p22009_a9,participant.p22009_a10
0,1000639,1,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,...,21.30840,-14.41990,36.33220,-68.67570,6.79464,2.918310,34.677700,20.404300,0.293947,1.722900
1,1001142,0,55.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,...,-1.76094,-10.55370,-6.37599,-2.46630,-5.08631,-3.109580,4.386370,3.252660,-3.096380,-2.796150
2,1001251,0,NaN,NaN,NaN,NaN,0.0,1.0,NaN,0.0,...,-14.24570,3.96257,-1.01864,2.27742,-4.21549,-0.100448,1.524450,-1.246880,1.368960,2.570260
3,1001445,1,60.0,NaN,NaN,NaN,1.0,1.0,NaN,1.0,...,-13.43690,3.02070,-3.19577,0.42849,6.21226,-1.478700,1.282360,-1.606150,0.199501,-4.308460
4,1002533,0,40.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,...,-13.81500,2.60585,-5.53910,8.82807,22.86970,2.332280,0.505821,0.470971,1.502100,0.467284


In [21]:
case_df = case_df.rename(columns=lambda x: re.sub('participant.', '', x))
case_df.head()

,eid,p31,p2966_i0,p2966_i1,p2966_i2,p2966_i3,p22001,p22006,p22019,p22021,...,p22009_a1,p22009_a2,p22009_a3,p22009_a4,p22009_a5,p22009_a6,p22009_a7,p22009_a8,p22009_a9,p22009_a10
0,1000639,1,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,...,21.30840,-14.41990,36.33220,-68.67570,6.79464,2.918310,34.677700,20.404300,0.293947,1.722900
1,1001142,0,55.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,...,-1.76094,-10.55370,-6.37599,-2.46630,-5.08631,-3.109580,4.386370,3.252660,-3.096380,-2.796150
2,1001251,0,NaN,NaN,NaN,NaN,0.0,1.0,NaN,0.0,...,-14.24570,3.96257,-1.01864,2.27742,-4.21549,-0.100448,1.524450,-1.246880,1.368960,2.570260
3,1001445,1,60.0,NaN,NaN,NaN,1.0,1.0,NaN,1.0,...,-13.43690,3.02070,-3.19577,0.42849,6.21226,-1.478700,1.282360,-1.606150,0.199501,-4.308460
4,1002533,0,40.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,...,-13.81500,2.60585,-5.53910,8.82807,22.86970,2.332280,0.505821,0.470971,1.502100,0.467284


Create phenotype variable and concatenate cohorts into one dataframe.

In [22]:
case_df['ischemia_cc'] = 1
control_df['ischemia_cc'] = 0

In [23]:
df = pd.concat([case_df, control_df])

In [24]:
# Note: The counts should be consistent with the counts from the Cohort Browser
df.ischemia_cc.value_counts()

0    400946
1     91331
Name: ischemia_cc, dtype: int64

Here is an example of retrieved data.

*Note: this table contains synthetic data and is made for demonstration purposes only!*

|Row|eid|p31|p2966_i0|p2966_i1|p2966_i2|p2966_i3|p22001|p22006|p22019|p22021|p22022|p23104_i0|p20160_i0|p30760_i0|p30780_i0|p22020|ischemia_cc|
|:---|:--- |:---|:--- |:---|:---|:---|:---|:--- |:---|:---|:--- |:---|:--- |:---|:--- |:---|:---|
|0 |EID-XXXXXXX|0 |58.0|NA|NA|NA|0.0|1.0|NA|0.0|61.0|28.5|1.0|0.959|2.456|1.0|1|
|1 |EID-YYYYYYY|1 |NA  |NA|NA|NA|1.0|1.0|NA|0.0|51.0|24.5|1.0|1.159|2.719|1.0|1|
|2 |EID-ZZZZZZZ|1 |50.0|NA|NA|NA|1.0|1.0|NA|1.0|64.0|29.7|1.0|0.970|3.440|NA |1|

In [25]:
print(df.shape)
df.head()

(492277, 27)


,eid,p31,p2966_i0,p2966_i1,p2966_i2,p2966_i3,p22001,p22006,p22019,p22021,...,p22009_a2,p22009_a3,p22009_a4,p22009_a5,p22009_a6,p22009_a7,p22009_a8,p22009_a9,p22009_a10,ischemia_cc
0,1000639,1,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,...,-14.41990,36.33220,-68.67570,6.79464,2.918310,34.677700,20.404300,0.293947,1.722900,1
1,1001142,0,55.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,...,-10.55370,-6.37599,-2.46630,-5.08631,-3.109580,4.386370,3.252660,-3.096380,-2.796150,1
2,1001251,0,NaN,NaN,NaN,NaN,0.0,1.0,NaN,0.0,...,3.96257,-1.01864,2.27742,-4.21549,-0.100448,1.524450,-1.246880,1.368960,2.570260,1
3,1001445,1,60.0,NaN,NaN,NaN,1.0,1.0,NaN,1.0,...,3.02070,-3.19577,0.42849,6.21226,-1.478700,1.282360,-1.606150,0.199501,-4.308460,1
4,1002533,0,40.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,...,2.60585,-5.53910,8.82807,22.86970,2.332280,0.505821,0.470971,1.502100,0.467284,1


## QC samples based on several conditions

In [26]:
df_qced = df[
    (df['p31'] == df['p22001']) # Filter reported sex and genetic sex are the same
    & (df['p22006'] == 1)  # Only include Caucasian ancestry (In_white_british_ancestry_subset)
    & (  
        df['p22019'].isnull() # No Sex chromosome aneuploidy
    )
    & (  
        df['p22020'] == 1 # Participant was used to calculate PCA (only non-relatives were included)
    )  
].copy()

In [27]:
df_qced.ischemia_cc.value_counts()

0    273298
1     58698
Name: ischemia_cc, dtype: int64

In [28]:
df_qced.head()

,eid,p31,p2966_i0,p2966_i1,p2966_i2,p2966_i3,p22001,p22006,p22019,p22021,...,p22009_a2,p22009_a3,p22009_a4,p22009_a5,p22009_a6,p22009_a7,p22009_a8,p22009_a9,p22009_a10,ischemia_cc
2,1001251,0,NaN,NaN,NaN,NaN,0.0,1.0,NaN,0.0,...,3.96257,-1.018640,2.277420,-4.21549,-0.100448,1.524450,-1.246880,1.368960,2.570260,1
3,1001445,1,60.0,NaN,NaN,NaN,1.0,1.0,NaN,1.0,...,3.02070,-3.195770,0.428490,6.21226,-1.478700,1.282360,-1.606150,0.199501,-4.308460,1
5,1003040,0,54.0,NaN,NaN,NaN,0.0,1.0,NaN,1.0,...,4.63916,-2.339450,0.502712,-2.63828,0.662364,0.931805,0.987020,-5.069820,2.115390,1
8,1004201,0,NaN,NaN,NaN,NaN,0.0,1.0,NaN,1.0,...,3.71645,-0.737745,1.572150,-1.20120,-1.681440,1.290020,-0.223860,-5.693960,-1.878020,1
9,1004219,0,28.0,NaN,NaN,NaN,0.0,1.0,NaN,0.0,...,2.01404,-2.064440,1.280780,-1.68080,2.864480,2.142890,-0.380478,-4.603690,0.058457,1


## Prepare covariates for subsequent regression analysis

Combine all instances of `Age high blood pressure diagnosed` (`2966`) field and make `hypertension` boolean variable.

In [29]:
df_qced['hypertension'] = [
    np.nansum([float(row[i]) for i in [0, 1, 2, 3]]) > 0
    for row in df_qced[['p2966_i0', 'p2966_i1', 'p2966_i2', 'p2966_i3']].to_numpy()
]
df_qced['hypertension'] = df_qced['hypertension'].astype(int)

For the `Ever smoked` (`20160`) make all missing values as `0`.

In [30]:
df_qced['p20160_i0'].fillna(0, inplace=True)

For the rest of covariate columns, replace missing values by mean for that column.

In [31]:
df_qced['p23104_i0'].fillna(df_qced['p23104_i0'].mean(), inplace=True)
df_qced['p30760_i0'].fillna(df_qced['p30760_i0'].mean(), inplace=True)
df_qced['p30780_i0'].fillna(df_qced['p30780_i0'].mean(), inplace=True)

In [32]:
df_qced.isna().sum()

eid                  0
p31                  0
p2966_i0        243046
p2966_i1        327766
p2966_i2        317869
p2966_i3        330814
p22001               0
p22006               0
p22019          331996
p22021               0
p21022               0
p23104_i0            0
p20160_i0            0
p30760_i0            0
p30780_i0            0
p22020               0
p22009_a1            0
p22009_a2            0
p22009_a3            0
p22009_a4            0
p22009_a5            0
p22009_a6            0
p22009_a7            0
p22009_a8            0
p22009_a9            0
p22009_a10           0
ischemia_cc          0
hypertension         0
dtype: int64

## Rename columns and organize it in format suitable for PLINK and regenie

In [33]:
# Rename columns for better readibility
df_qced = df_qced.rename(columns=lambda x: re.sub('p22009_a','pc',x))
df_qced = df_qced.rename(
    columns={
        'eid': 'IID',
        'p31': 'sex',
        'p21022': 'age',
        'p20160_i0': 'ever_smoked',
        'p23104_i0': 'bmi',
        'p30760_i0': 'hdl_cholesterol',
        'p30780_i0': 'ldl_cholesterol',
    }
)

# Add FID column -- required input format for regenie
df_qced['FID'] = df_qced['IID']

# Create a phenotype table from our QCed data
cols = [
        'FID',
        'IID',
        'sex',
        'age',
        'bmi',
        'ever_smoked',
        'hdl_cholesterol',
        'ldl_cholesterol',
        'hypertension',
        'ischemia_cc',
]
cols.extend([col for col in df_qced if 'pc' in col])
df_phenotype = df_qced[cols]

In [34]:
df_phenotype.head()

,FID,IID,sex,age,bmi,ever_smoked,hdl_cholesterol,ldl_cholesterol,hypertension,ischemia_cc,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10
2,1001251,1001251,0,51,24.4,1.0,1.452362,3.570649,0,1,-14.2457,3.96257,-1.018640,2.277420,-4.21549,-0.100448,1.524450,-1.246880,1.368960,2.570260
3,1001445,1001445,1,63,30.2,1.0,0.973000,2.930000,1,1,-13.4369,3.02070,-3.195770,0.428490,6.21226,-1.478700,1.282360,-1.606150,0.199501,-4.308460
5,1003040,1003040,0,57,31.7,1.0,1.291000,2.345000,1,1,-13.4688,4.63916,-2.339450,0.502712,-2.63828,0.662364,0.931805,0.987020,-5.069820,2.115390
8,1004201,1004201,0,55,36.8,1.0,1.465000,4.716000,0,1,-11.7176,3.71645,-0.737745,1.572150,-1.20120,-1.681440,1.290020,-0.223860,-5.693960,-1.878020
9,1004219,1004219,0,45,37.1,0.0,1.483000,3.084000,1,1,-12.1573,2.01404,-2.064440,1.280780,-1.68080,2.864480,2.142890,-0.380478,-4.603690,0.058457


## Select only samples that have imputed data available and save phenotype table as CSV

In [35]:
# Get imputed data
path_to_impute_file = f'/mnt/project/Bulk/Imputation/{imputation_folder}/ukb{imputation_field_id}_c1_b0_v1.sample'
sample_file = pd.read_csv(
    path_to_impute_file,
    delimiter='\s',
    header=0,
    names=['FID', 'IID', 'missing', 'sex'],
    engine='python',
)
# Intersect the phenotype file and the imputed .sample file
# to generate phenotype DataFrame for only samples included in the imputed data
ischemia_df = df_phenotype.join(
    sample_file.set_index('IID'), on='IID', rsuffix='_sample', how='inner'
)
# Drop unuseful columns from .fam file
ischemia_df.drop(
    columns=['FID_sample', 'missing', 'sex_sample'],
    axis=1,
    inplace=True,
    errors='ignore',
)

In [36]:
# Write phenotype files to a TSV file
ischemia_df.to_csv('ischemia_df.phe', sep='\t', na_rep='NA', index=False, quoting=3)

In [37]:
ischemia_df.head()

,FID,IID,sex,age,bmi,ever_smoked,hdl_cholesterol,ldl_cholesterol,hypertension,ischemia_cc,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10
2,1001251,1001251,0,51,24.4,1.0,1.452362,3.570649,0,1,-14.2457,3.96257,-1.018640,2.277420,-4.21549,-0.100448,1.524450,-1.246880,1.368960,2.570260
3,1001445,1001445,1,63,30.2,1.0,0.973000,2.930000,1,1,-13.4369,3.02070,-3.195770,0.428490,6.21226,-1.478700,1.282360,-1.606150,0.199501,-4.308460
5,1003040,1003040,0,57,31.7,1.0,1.291000,2.345000,1,1,-13.4688,4.63916,-2.339450,0.502712,-2.63828,0.662364,0.931805,0.987020,-5.069820,2.115390
8,1004201,1004201,0,55,36.8,1.0,1.465000,4.716000,0,1,-11.7176,3.71645,-0.737745,1.572150,-1.20120,-1.681440,1.290020,-0.223860,-5.693960,-1.878020
9,1004219,1004219,0,45,37.1,0.0,1.483000,3.084000,1,1,-12.1573,2.01404,-2.064440,1.280780,-1.68080,2.864480,2.142890,-0.380478,-4.603690,0.058457


## Load file to project storage

In [38]:
%%bash -s "$output_dir"
# Upload the geno-pheno intersect phenotype file back to the RAP project
dx upload ischemia_df.phe -p --path $1 --brief

file-J0G5yy0JjKYBzvP4Vx26PbG0


Here is an example of phenotype file.

*Note: this table contains synthetic data and is made for demonstration purposes only!*

|Row|FID|IID|sex|age|bmi|ever_smoked|hypertension|hdl_cholesterol|ldl_cholesterol|ischemia_cc|
|:--- |:--- |:---|:--- |:---|:---|:---|:---|:--- |:---|:---|
|0|EID-XXXXXXX|1234567|0|61.0|28.5|0|1|0.959|2.456|1|1|1|
|1|EID-YYYYYYY|1234568|1|51.0|24.5|0|0|1.159|2.719|0|1|1|
|2|EID-ZZZZZZZ|1234569|0|60.0|23.8|1|1|1.579|3.396|0|1|1|

## Output files

- Table containing phenotype and covariates to be used in regenie GWAS analysis (`ischemia_df.phe`)